# 🐩 Poodle Palette – Genotype → Phenotype Tutor
Welcome! In this notebook you’ll learn how two key genes (B and E) create the classic Poodle coat colours—and you’ll see the result for *any* genotype you enter.

In [1]:
# 📦 Setup
# If running locally, make sure pandas and matplotlib are installed.
# In Google Colab these come pre‑installed.
import pandas as pd
import matplotlib.pyplot as plt

# Display plots inline
%matplotlib inline


## 🎓 2. Genetic Background
Poodle colours hinge mainly on two loci:

| Locus | Dominant / recessive | What it controls |
|-------|----------------------|------------------|
| **B** | **B** = black pigment<br>**b** = brown pigment | `bb` turns all black areas brown (“liver”). |
| **E** | **E** = allows black/brown<br>**e** = blocks black → red/cream | `ee` dogs show only red/cream pigment. |

*We’ll ignore rarer loci (Dilute, Spotting) to keep the demo bite-sized.*

In [11]:
## 3. Imports + Helper Dictionaries


In [25]:
# 📦 Standard data-science imports
import pandas as pd
import matplotlib.pyplot as plt

# 📚 Teaching widgets (for the interactive allele picker we’ll add later)
from IPython.display import HTML
import ipywidgets as widgets

# 🖼️ Show charts inside the notebook
%matplotlib inline

# ────────────────────────────────────────────────────────────────
# Helper dictionaries for quick look-ups at each locus
# These will be used by predict_colour() in the next section.

# B-locus (black vs. brown pigment)
B_LOCUS = {
    "BB": "Black",
    "Bb": "Black",
    "bb": "Brown"
}

# E-locus (extension / red-cream override)
E_LOCUS = {
    "EE": "Allows black/brown",
    "Ee": "Allows black/brown",
    "ee": "Red/Cream (blocks black)"}

# Spotting / Parti – dominant solid S, recessive parti sp
S_LOCUS = {
    "SS": "solid",
    "Ssp": "solid (carrier)",
    "spsp": "parti"
}

# K-locus hierarchy  KB  >  Kbr  >  ky
K_HIERARCHY = ["KB", "Kbr", "ky"]    # first one present wins

# A-locus hierarchy  ay  >  at  >  aw  >  a
A_HIERARCHY = ["ay", "at", "aw", "a"]


In [14]:
## 4. Function `predict_colour()`

In [42]:
# ── dominance tables ──────────────────────────────────────────────
K_HIERARCHY = ["Kbr", "KB"]          # check Kbr first
A_HIERARCHY = ["ay", "at", "aw", "a"]  # ay dominates at, etc.

def predict_colour(b_pair: str, e_pair: str,
                   k_pair: str,
                   a1: str, a2: str,          # <-- split A alleles
                   s_pair: str) -> str:
    """
    Return a plain-English Poodle coat description from:
      B_pair, E_pair, K_pair, A1, A2, S_pair
    """

    # 1️⃣  Base pigment (B & E)
    if e_pair.islower():                            # 'ee'
        nose = "brown" if b_pair.islower() else "black"
        colour = f"Red/Cream (nose: {nose})"
    else:
        colour = "Brown" if b_pair.islower() else "Black"

    # 2️⃣  Determine dominant K allele
    if "Kbr" in k_pair:
        k_dom = "Kbr"
    elif "KB" in k_pair:
        k_dom = "KB"
    else:
        k_dom = "ky"

    # 3️⃣  Determine dominant A allele
    a_pair = a1 + a2             # e.g. "ayat"
    dom_a = next(a for a in A_HIERARCHY if a in a_pair)

    # 4️⃣  Pattern logic (K + A)
    if k_dom == "KB":
        pattern = ""
    elif k_dom == "Kbr":
        pattern = "" if dom_a == "a" else "Brindle "
    else:  # kyky
        pattern = {
            "ay": "Sable ",
            "at": "Phantom ",
            "aw": "Agouti ",
            "a":  ""
        }[dom_a]

    # 5️⃣  Spotting
    pattern_spot = "Parti " if s_pair.lower().endswith("spsp") else ""

    return f"{pattern_spot}{pattern}{colour}"


In [37]:
tests = [
    ("BB", "EE", "KBky", "ayat", "SS"),     # dominant black
    ("bb", "EE", "Kbrky", "ayat", "SS"),    # brindle sable (brown)
    ("BB", "EE", "kyky", "ataw", "spsp"),   # parti phantom
    ("bb", "EE", "kyky", "atay", "SS"),     #
    ("bb", "ee", "KBky", "ayay", "Ssp"),    # red/cream overrides
]
for t in tests:
    print(f"{t} → {predict_colour(*t)}")

('BB', 'EE', 'KBky', 'ayat', 'SS') → Black
('bb', 'EE', 'Kbrky', 'ayat', 'SS') → Brindle Brown
('BB', 'EE', 'kyky', 'ataw', 'spsp') → Parti Phantom Black
('bb', 'EE', 'kyky', 'atay', 'SS') → Sable Brown
('bb', 'ee', 'KBky', 'ayay', 'Ssp') → Parti Red/Cream (nose: brown)


In [8]:
## 5. Bulk Conversion of Your CSV

In [23]:
## 6. Interactive Playground

In [43]:
def show_interactive():
    # --- B locus ----------------------------------------------------------
    b_btn = widgets.ToggleButtons(
        options=[("BB (black)", "BB"),
                 ("Bb (black)", "Bb"),
                 ("bb (brown)", "bb")],
        description="B locus:",
    )

    # --- E locus ----------------------------------------------------------
    e_btn = widgets.ToggleButtons(
        options=[("EE (permits)", "EE"),
                 ("Ee (permits)", "Ee"),
                 ("ee (red/cream)", "ee")],
        description="E locus:",
    )

    # --- K locus ----------------------------------------------------------
    k_btn = widgets.ToggleButtons(
        options=[("KB/KB",  "KBKB"),
                 ("Kbr/ky", "Kbrky"),
                 ("ky/ky",  "kyky")],
        description="K locus:",
    )

    # --- A locus (two alleles) -------------------------------------------
    a1_btn = widgets.ToggleButtons(
        description="A-1:",
        options=[("ay", "ay"), ("at", "at"), ("aw", "aw"), ("a", "a")],
    )
    a2_btn = widgets.ToggleButtons(
        description="A-2:",
        options=[("ay", "ay"), ("at", "at"), ("aw", "aw"), ("a", "a")],
    )

    # --- Spotting ---------------------------------------------------------
    s_btn = widgets.ToggleButtons(
        description="S locus:",
        options=[("SS (solid)",  "SS"),
                 ("Ssp (carrier)", "Ssp"),
                 ("sp/sp (parti)", "spsp")],
    )

    # --------- live output area ------------------------------------------
    out = widgets.Output()

    def update(_=None):
        colour = predict_colour(
            b_btn.value,
            e_btn.value,
            k_btn.value,
            a1_btn.value,
            a2_btn.value,
            s_btn.value
        )
        with out:
            out.clear_output()
            display(HTML(f"<h3>{colour}</h3>"))

    # fire once at start
    update()

    # listen for *any* change
    for w in (b_btn, e_btn, k_btn, a1_btn, a2_btn, s_btn):
        w.observe(update, names="value")

    display(b_btn, e_btn, k_btn, a1_btn, a2_btn, s_btn, out)

# run it
show_interactive()


ToggleButtons(description='B locus:', options=(('BB (black)', 'BB'), ('Bb (black)', 'Bb'), ('bb (brown)', 'bb'…

ToggleButtons(description='E locus:', options=(('EE (permits)', 'EE'), ('Ee (permits)', 'Ee'), ('ee (red/cream…

ToggleButtons(description='K locus:', options=(('KB/KB', 'KBKB'), ('Kbr/ky', 'Kbrky'), ('ky/ky', 'kyky')), val…

ToggleButtons(description='A-1:', options=(('ay', 'ay'), ('at', 'at'), ('aw', 'aw'), ('a', 'a')), value='ay')

ToggleButtons(description='A-2:', options=(('ay', 'ay'), ('at', 'at'), ('aw', 'aw'), ('a', 'a')), value='ay')

ToggleButtons(description='S locus:', options=(('SS (solid)', 'SS'), ('Ssp (carrier)', 'Ssp'), ('sp/sp (parti)…

Output()

In [ ]:
## 7. Visual Summary

In [10]:
## 8. Further Reading